In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

# Ensure inline plotting for Jupyter Lab
%matplotlib inline

# Set the base file path
file_path = 'F:/SCANS/0-ME-Collagen Project/1- Bruce in vivos/CTFire/ROIS/output/ctFIREout'

# Generate patterns for DIF and PIF from 1 to 11
patterns = [f"/HistANG_ctFIRE_DIF{i}_Colour_*.csv" for i in range(1, 12)]
patterns += [f"/HistANG_ctFIRE_PIF{i}_Colour_*.csv" for i in range(1, 12)]

# Find all matching files for each pattern
matching_files = []
for pattern in patterns:
    matching_files += glob.glob(file_path + pattern)

# Load CSV files, add a column for file type, and concatenate
df_list = []
for file in matching_files:
    temp_df = pd.read_csv(file, header=None, names=['Angles'])
    file_type = os.path.basename(file).split('_')[2]
    temp_df['Type'] = file_type
    df_list.append(temp_df)

# Concatenate all dataframes
df = pd.concat(df_list, ignore_index=True)

# Function to filter data based on file type
def filter_data(df, type_pattern):
    return df[df['Type'].str.contains(type_pattern)]

# Filter data for PIF and DIF
df_pif = filter_data(df, 'PIF')
df_dif = filter_data(df, 'DIF')

# Function to calculate histogram and SEM
def calculate_histogram_and_sem(data, bins):
    hist, bin_edges = np.histogram(data.dropna(), bins=bins, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    hist = hist * np.diff(bin_edges) * 100  # Convert to percentages

    # Calculate SEM for each bin
    sems = []
    for i in range(len(bin_edges) - 1):
        bin_data = data[(data >= bin_edges[i]) & (data < bin_edges[i + 1])]
        sem = bin_data.std() / np.sqrt(len(bin_data)) if len(bin_data) > 0 else 0
        sems.append(sem)

    return hist, bin_centers, sems

# Set bins and calculate histogram and SEM
bins = np.arange(0, 181, 10)
hist_pif, bin_centers_pif, sem_pif = calculate_histogram_and_sem(df_pif['Angles'], bins)
hist_dif, bin_centers_dif, sem_dif = calculate_histogram_and_sem(df_dif['Angles'], bins)

# Convert dimensions from cm to inches
width_in_inches = 4 / 2.54
height_in_inches = 3 / 2.54

# Create figure with adjusted font sizes and layout
plt.figure(figsize=(width_in_inches, height_in_inches))
plt.rcParams.update({'font.size': 7})

# Plot error bars with line plots
plt.errorbar(bin_centers_pif, hist_pif, yerr=sem_pif, fmt='-', capsize=3, label='PIF', color='lightcoral', linewidth=1.5, ecolor='darkred')
plt.errorbar(bin_centers_dif, hist_dif, yerr=sem_dif, fmt='-', capsize=3, label='DIF', color='lightgreen', linewidth=1.5, ecolor='darkgreen')

# Add grid and labels
plt.xlabel('Angle Deviation (°)', fontsize=8)
plt.ylabel('Percentage (%)', fontsize=8)
plt.title('Angle Deviations Between DIF and PIF', fontsize=8)
plt.legend(loc='upper right', fontsize=6)
plt.grid(True, linestyle='--', linewidth=0.5)

# Set ticks and adjust layout
plt.xticks(ticks=np.arange(0, 181, 20), fontsize=7)
plt.yticks(fontsize=7)
plt.tight_layout()

# Display the plot
plt.show()
